<a href="https://colab.research.google.com/github/riminipa16/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [48]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [49]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [51]:
#loading the dataset to a pandas Dataframe
news_dataset = pd.read_csv('/content/news.csv')

In [52]:
news_dataset.shape

(6335, 4)

In [53]:
# print the first 5 rows of the Dataframe
news_dataset.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [54]:
# counting the missing values in the Dataset
news_dataset.isnull().sum()

,0
id,0
title,0
text,0
label,0


In [55]:
# seperating the data and labels
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [56]:
print(X)
print(Y)

         id  ...                                               text
0      8476  ...  Daniel Greenfield, a Shillman Journalism Fello...
1     10294  ...  Google Pinterest Digg Linkedin Reddit Stumbleu...
2      3608  ...  U.S. Secretary of State John F. Kerry said Mon...
3     10142  ...  — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       875  ...  It's primary day in New York and front-runners...
...     ...  ...                                                ...
6330   4490  ...  The State Department told the Republican Natio...
6331   8062  ...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332   8622  ...   Anti-Trump Protesters Are Tools of the Oligar...
6333   4021  ...  ADDIS ABABA, Ethiopia —President Obama convene...
6334   4330  ...  Jeb Bush Is Suddenly Attacking Trump. Here's W...

[6335 rows x 3 columns]
0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Len

Stemming:

 Stemming is  the process of reducing a word to its Root word

 example:

 actor, acting, actress -->act

In [57]:
# Initialize stemmer
port_stem = PorterStemmer()

In [61]:
# Preprocessing function
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [63]:
news_dataset['text'] = news_dataset['text'].apply(stemming)

In [64]:
print(news_dataset['text'])

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: text, Length: 6335, dtype: object


In [66]:
# seperating the data and label
X = news_dataset['text']
Y = news_dataset['label']

In [67]:
print(X)

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: text, Length: 6335, dtype: object


In [68]:
print(Y)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object


In [69]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [70]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1598597 stored elements and shape (6335, 43733)>
  Coords	Values
  (0, 106)	0.02761351227547292
  (0, 212)	0.023325671822305483
  (0, 271)	0.054392547007153
  (0, 328)	0.060358339691341766
  (0, 357)	0.013921754610837712
  (0, 451)	0.020578335403474236
  (0, 578)	0.07906058794712593
  (0, 617)	0.019850889079147057
  (0, 620)	0.01867769280554434
  (0, 622)	0.044611051373187424
  (0, 648)	0.015493583340569357
  (0, 680)	0.019224695171899866
  (0, 978)	0.0267316264615063
  (0, 999)	0.020738114075644642
  (0, 1022)	0.0371485992478585
  (0, 1033)	0.014808690907292291
  (0, 1085)	0.028815013247480384
  (0, 1089)	0.018102417756284926
  (0, 1197)	0.022505695388032227
  (0, 1206)	0.010675511636980443
  (0, 1549)	0.02507693986049084
  (0, 1665)	0.025967702221880915
  (0, 1713)	0.041128098634750046
  (0, 1744)	0.027618563562476325
  (0, 1783)	0.018872240981032625
  :	:
  (6334, 39650)	0.02398512016423436
  (6334, 39971)	0.0407182753213

Splitting the Dataset to training and test data

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

Training The Model : Logistic Regression

In [71]:
model = LogisticRegression()

In [73]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [74]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)


In [75]:
print('Accuracy score of the training data: ', training_data_accuracy)

Accuracy score of the training data:  0.9518547750591949


In [76]:
# Accuracy on test data
test_accuracy = accuracy_score(Y_test, model.predict(X_test))
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9163378058405682


Make a predictive system

In [82]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if prediction[0] == 0:
  print('The news is Real')
else:
  print('The news is Fake')



['REAL']
The news is Fake
